<a href="https://colab.research.google.com/github/RiyaJohn/HE_IndiaMartPriceRangePrediction/blob/master/PriceRangePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
import pandas as pd
from scipy import stats
import numpy as np
import sklearn
from collections import Counter
import matplotlib.pyplot as plt

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Data Cleaning


In [73]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Colab Notebooks/data/Unit Quantity Codes.xlsx"
!ls "/content/drive/My Drive/Colab Notebooks/data/Phase 1 Dataset Hackathon (1)d09fdc4.xlsx"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'/content/drive/My Drive/Colab Notebooks/data/Unit Quantity Codes.xlsx'
'/content/drive/My Drive/Colab Notebooks/data/Phase 1 Dataset Hackathon (1)d09fdc4.xlsx'


In [82]:
# Get Static dataset of Indian Customs Quantity Codes and Shipping Measurement Units
qty_codes_file = "/content/drive/My Drive/Colab Notebooks/data/Unit Quantity Codes.xlsx"
qty_codes = pd.read_excel(qty_codes_file)
qty_codes = pd.DataFrame(qty_codes)
qty_codes.head(5)

,Quantity Code,Quantity Description,Quantity Type
0,AMP,AMPULES,NaN
1,BAG,BAGS,NaN
2,BAL,BALE,M
3,BDL,BUNDLES,NaN
4,BKL,BUCKLES,M


In [0]:
# Data collection
data_file = "/content/drive/My Drive/Colab Notebooks/data/Phase 1 Dataset Hackathon (1)d09fdc4.xlsx"
data = pd.read_excel(data_file, None) # None - Reading all sheets in the file

In [0]:
# Remove NAs and zeros data
data_dfs = []
for key, value in data.items():
  df = pd.DataFrame(data[key])
  df = df.dropna() # remove NAs
  df = df[(df != 0).all(1)] # remove rows that have zeros in any of the col.
  data_dfs.append(df)

### Data Processing

In [0]:
# getting the unique set of units in the input data
input_units = []
for df in data_dfs:
  df['Unit'] = df['Unit'].astype(str).str.upper()
  input_units.append(df.Unit.unique().tolist())
input_units = set([item for sublist in input_units for item in sublist])

In [86]:
# Getting a matching std. unit code for input units
input_units_to_codes = {}

for input_unit in input_units:
  unit_code = "" # []
  unit_scr = 100
  for index, row in qty_codes.iterrows():
    scr1 = nltk.edit_distance(input_unit, row[0])
    scr2 = nltk.edit_distance(input_unit, row[1])
    scr = scr1 if scr1 < scr2 else scr2
    if scr < unit_scr:
      unit_code = row[0]
      unit_scr = scr
  if unit_scr <=3:
    input_units_to_codes[input_unit] = unit_code
print(input_units_to_codes)

{'PEICE': 'PAC', 'PIECES': 'PCS', 'PACKET': 'PAC', 'CARTON': 'CTN', 'DAY': 'BAG', 'PC': 'PAC', '1 PCS': 'PCS', 'PAIR(S)': 'PRS', 'METER': 'MTR', 'BAG': 'BAG', '4 PCS': 'PCS', '1PIECE': 'PCS', 'NUMBER': 'NOS', 'FEET': 'FTS', 'PIECE': 'PCS', '12 UNITS': 'UNT', '5': 'AMP', 'SETS': 'SET', 'BARREL': 'BAL', 'OUNCE': 'BUN', '1PCD': 'PCS', 'PCS': 'PCS', 'YEAR': 'BAG', '1 PIECE': 'PCS', 'ONWARDS': 'YDS', 'SUIT': 'KIT', '3 SET': 'SET', '1PIS': 'PCS', 'DOLLAR': 'RLS', 'PAIR': 'PRS', 'PICES': 'PCS', 'SET': 'SET', '1 PC': 'AMP', 'KURTI': 'CRT', 'PICS': 'PCS', 'SET(S)': 'SET', 'PIECE(S)': 'PCS', 'BOX': 'BOX', 'KILOGRAM': 'KGS', 'POUND': 'LBS', 'UNIT': 'UNT', 'NO': 'NOS', 'PSC': 'PAC', '1PC': 'PAC', 'IN': 'BUN', '4 UNITS': 'UNT', 'RS': 'GRS', '1': 'AMP', '1PCS': 'PCS', 'GRAM': 'GMS', 'ONE': 'KME', 'UNIT(S)': 'UNT', 'PACK': 'PAC', 'KIT': 'KIT', '1NOS': 'NOS', '10': 'AMP', 'PI': 'KIT'}


In [0]:
# Replacing input units with matching std. unit code/s
for df in data_dfs:
  df['Unit'] = df['Unit'].map(input_units_to_codes)

In [88]:
res = {}
for df in data_dfs:
  item = df['Category Name'].unique()[0]
  res[item] = {}
  total_cnt = len(df)
  grouped_df = df.groupby('Unit')
  for name, group in grouped_df: 
    group = group[(np.abs(stats.zscore(group.Price)) < 5)] # Remove rows with outliers(+/-3 std. deviation away) in Price column
    if len(group) > 3:
      is_acc = True if len(group) > total_cnt/10 else False
      unit = name
      skew_score = group.Price.skew()
      if skew_score > 1 or skew_score < -1:
        log_data = np.log(group.Price)
        price_range = log_data.quantile([0.1,1])
        res[item][unit] = (round(np.exp(price_range[0.1])), round(np.exp(price_range[1])), is_acc)
      else:
        price_range = group.Price.quantile([0.1,1])
        res[item][unit] = (round(price_range[0.1]), round(price_range[1]), is_acc)


/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:2315: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [89]:
for item, prices in res.items():
  for unit, price_range in prices.items():
    unit_desc = qty_codes.loc[qty_codes['Quantity Code'] == unit].iloc[0]['Quantity Description']
    if price_range[2] == False:
      print("The price of {} per {} ranges from Rs.{} to Rs. {}.(with less data hence less accuracy)".format(item, unit_desc, price_range[0], price_range[1] ))
    else: 
      print("The price of {} per {} ranges from Rs.{} to Rs. {}.".format(item, unit_desc, price_range[0], price_range[1] ))
   
    

The price of Leather Safety Gloves per PACKS ranges from Rs.69.0 to Rs. 18974.0.(with less data hence less accuracy)
The price of Leather Safety Gloves per PIECES ranges from Rs.40.0 to Rs. 695.0.(with less data hence less accuracy)
The price of Leather Safety Gloves per PAIRS ranges from Rs.33.0 to Rs. 450.0.
The price of Leather Safety Gloves per UNITS ranges from Rs.114.0 to Rs. 8978.0.(with less data hence less accuracy)
The price of Ladies Kurta per AMPULES ranges from Rs.199.0 to Rs. 3222.0.(with less data hence less accuracy)
The price of Ladies Kurta per BAGS ranges from Rs.287.0 to Rs. 4792.0.(with less data hence less accuracy)
The price of Ladies Kurta per BOX ranges from Rs.206.0 to Rs. 2350.0.(with less data hence less accuracy)
The price of Ladies Kurta per BUNCHES ranges from Rs.409.0 to Rs. 1650.0.(with less data hence less accuracy)
The price of Ladies Kurta per CARAT ranges from Rs.449.0 to Rs. 700.0.(with less data hence less accuracy)
The price of Ladies Kurta per K